In [1]:
# Main notebook for playing around looking at different things in data
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
# plt.rcParams["image.cmap"] = "Dark2"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
plt.rcParams['lines.markersize'] = 10
plt.rcParams['lines.linewidth'] = 3
%matplotlib widget
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]


saveFigures is set to: True
Done loading packages


In [2]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath


# for subdir, dirs, files in os.walk(rootdir):
#     if not len(files) == 0:
#         latestdir = subdir
#         latestDate = pd.to_datetime(subdir[-10:])

# print(latestdir)
# print(latestDate)

latestdir = rootdir+ '/SSI_data_2021-04-29'
latestDate = pd.to_datetime(latestdir[-10:])

print(latestdir)
print(latestDate)

c:\Users\rakrpe\Documents\PandemiX\GithubRepos\PandemiX\DanskeData/ssi_data/SSI_data_2021-04-29
2021-04-29 00:00:00


In [3]:
# Load the data
fulldfPCR = pd.read_csv(latestdir+'/Test_pos_over_time.csv',delimiter = ';',dtype=str)
fulldf = pd.read_csv(latestdir+'/Test_pos_over_time_antigen.csv',delimiter = ';',dtype=str)

# # Cut out the last two rows of summary numbers
dfPCR = fulldfPCR.iloc[:-2,:].copy()
df = fulldf.iloc[:-2,:].copy()

In [4]:
dfPCR["PosPct"] = pd.to_numeric(dfPCR["PosPct"].astype(str).apply(lambda x: x.replace(',','.')))
dfPCR["Date"] = dfPCR["Date"].astype('datetime64[D]')
df["PosPct"] = pd.to_numeric(df["PosPct"].astype(str).apply(lambda x: x.replace(',','.')))
df["Date"] = df["Date"].astype('datetime64[D]')

rows_to_fix_period_in = ["NewPositive","NotPrevPos","PrevPos","Tested","Tested_kumulativ"]
for name in rows_to_fix_period_in:
    df[name] = pd.to_numeric(df[name].astype(str).apply(lambda x: x.replace('.','')))
    dfPCR[name] = pd.to_numeric(dfPCR[name].astype(str).apply(lambda x: x.replace('.','')))


In [5]:
# Load the data

df2 = pd.read_csv(latestdir+'/Antigentests_pr_dag.csv',delimiter = ';')

# # Remove the first two rows which are from april 2020
# df2 = df2.iloc[2:,:].copy()

In [6]:
# df2

In [7]:
df2["Dato"] = df2["Dato"].astype('datetime64[D]')
df2["Andel_AGpos_PCRkonf_pct"] = pd.to_numeric(df2["Andel_AGpos_PCRkonf_pct"].astype(str).apply(lambda x: x.replace(',','.')))
df2["Andel_AGminusPCRkonf_pct"] = pd.to_numeric(df2["Andel_AGminusPCRkonf_pct"].astype(str).apply(lambda x: x.replace(',','.')))

In [8]:
# print(df2.columns.values)
# df2.tail()
df2

,Dato,AG_testede,AG_pos,AGpos_PCRkonf,Andel_AGpos_PCRkonf_pct,AGpos_minusPCRkonf,Andel_AGminusPCRkonf_pct,AGpos_PCRpos,AGposPCRneg,AGnegPCRpos,AGnegPCRneg
0,2021-02-01,4827,8,4,50.000000,4,50.000000,2,2,2,293
1,2021-02-02,4640,9,5,55.555556,4,44.444444,5,0,5,308
2,2021-02-03,4869,10,6,60.000000,4,40.000000,6,0,6,343
3,2021-02-04,4494,6,3,50.000000,3,50.000000,3,0,5,322
4,2021-02-05,4360,7,5,71.428571,2,28.571429,4,1,6,280
5,2021-02-06,2586,10,6,60.000000,4,40.000000,6,0,3,262
6,2021-02-07,3020,14,7,50.000000,7,50.000000,7,0,6,326
7,2021-02-08,6979,13,7,53.846154,6,46.153846,6,1,5,439
8,2021-02-09,6086,6,4,66.666667,2,33.333333,4,0,5,390
9,2021-02-10,7070,13,5,38.461538,8,61.538462,4,1,1,498


In [9]:
# plt.figure()
# # plt.plot(df2.Dato,df2.AG_testede)
# # plt.plot(df2.Dato,df2.Andel_AGpos_PCRkonf_pct)
# plt.plot(df2.Dato,df2.AG_pos)
# # plt.plot(df2.Dato,df2.AGpos_PCRkonf)
# plt.plot(df2.Dato,df2.AGpos_PCRkonf+df2.AGpos_minusPCRkonf)
# # plt.plot(df2.Dato,df2.AGpos_PCRpos)
# # plt.plot(df2.Dato,df2.AGposPCRneg)
# # plt.plot(df2.Dato,df2.AGpos_PCRpos+df2.AGposPCRneg)
# plt.show()

In [10]:
# fig,(ax1,ax2) = plt.subplots(2,1)
fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)

ax1.fill_between(df2.Dato,df2.AG_pos,color='blue',label='AG pos, PCR testet')
ax1.fill_between(df2.Dato,df2.AGpos_minusPCRkonf,color='grey',label='AG pos, afventer PCR')

ax2.fill_between(df2.Dato,100*np.ones(df2.Dato.shape),color='xkcd:mint',label='AG pos, PCR neg')
ax2.fill_between(df2.Dato,100*np.divide(df2.AGpos_PCRpos,df2.AGpos_PCRkonf),color='g',label='AG pos, PCR pos')

numAGnegKonf = df2.AGnegPCRpos + df2.AGnegPCRneg

ax3.fill_between(df2.Dato,100*np.ones(df2.Dato.shape),color='xkcd:mint',label='AG neg, PCR neg')
ax3.fill_between(df2.Dato,100*np.divide(df2.AGnegPCRpos,numAGnegKonf),color='g',label='AG neg, PCR pos')

ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax3.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=100)
ax3.set_ylim(bottom=0,top=3)
ax1.set_xlim([df2.Dato[0],df2.iloc[-1,0]])
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
df.tail()

,Date,NewPositive,NotPrevPos,PosPct,PrevPos,Tested,Tested_kumulativ
381,2021-04-24,176,162216,0.1,447,162663,7575404
382,2021-04-25,192,197290,0.1,576,197866,7773270
383,2021-04-26,351,362111,0.1,1008,363119,8136389
384,2021-04-27,294,287417,0.1,720,288137,8424526
385,2021-04-28,234,244103,0.1,673,244776,8669302


In [12]:
dfPCR.tail()

,Date,NewPositive,NotPrevPos,PosPct,PrevPos,Tested,Tested_kumulativ
453,2021-04-24,568,135092,0.4,4488,139580,26118778
454,2021-04-25,600,148021,0.4,4612,152633,26271411
455,2021-04-26,778,186135,0.4,7355,193490,26464901
456,2021-04-27,890,169291,0.5,6231,175522,26640423
457,2021-04-28,123,25979,0.5,1174,27153,26667576


In [13]:
df2.tail()

,Dato,AG_testede,AG_pos,AGpos_PCRkonf,Andel_AGpos_PCRkonf_pct,AGpos_minusPCRkonf,Andel_AGminusPCRkonf_pct,AGpos_PCRpos,AGposPCRneg,AGnegPCRpos,AGnegPCRneg
79,2021-04-21,265624,281,257,91.459075,24,8.540925,136,119,150,14051
80,2021-04-22,332245,261,239,91.570881,22,8.429119,114,123,124,16175
81,2021-04-23,293871,235,212,90.212766,23,9.787234,109,101,113,13702
82,2021-04-24,162736,196,168,85.714286,28,14.285714,90,77,119,8807
83,2021-04-25,197950,215,196,91.162791,19,8.837209,112,83,159,10324


In [14]:
fig,ax1 = plt.subplots(1,1)
ax1.plot(df.Date,100*df.Tested/5800000)
meanWidth=7
ax1.plot(rnTime(df.Date,meanWidth),100*rnMean(df.Tested,meanWidth)/5800000,'k:')

ax1.set_xlim([np.datetime64('2021-02-01'),df.iloc[-1,0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# fig,ax1 = plt.subplots(1,1)
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

ax1.plot(df.Date,df.Tested,label='Antal AG tests')
ax1.plot(dfPCR.Date,dfPCR.Tested,label='Antal PCR tests')
ax1.plot(rnTime(dfPCR.Date,7),rnMean(dfPCR.Tested,7),label='Antal PCR tests')

# ax2.plot(df.Date,df.PosPct,label='Positiv procent, AG')
# ax2.plot(dfPCR.Date,dfPCR.PosPct,label='Positive procent, PCR')
ax2.plot(df.Date,100*np.divide(df.NewPositive,df.Tested),label='Positiv procent, AG')
ax2.plot(dfPCR.Date,100*np.divide(dfPCR.NewPositive,dfPCR.Tested),label='Positiv procent, PCR')


ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

ax1.grid()
ax2.grid()

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=0.6)

ax1.set_xlim(left=np.datetime64('2020-02-01'),right=date.today())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d %b\n%Y'))

plt.tight_layout()

# if saveFigures:
#     plt.savefig('figs/AntalTestOgPosPct')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
firstDate = np.datetime64('2021-02-01')
lastDate = np.datetime64('2021-04-24')

curDates = np.arange(firstDate,lastDate)

# posAG  = df[df.Date.isin(curDates)].NewPositive 
PCRpos = dfPCR[dfPCR.Date.isin(curDates)].NewPositive.values
numPCR = dfPCR[dfPCR.Date.isin(curDates)].Tested.values
AGpos = df[df.Date.isin(curDates)].NewPositive.values
numAG = df[df.Date.isin(curDates)].Tested.values
AGposPCRpos = df2[df2.Dato.isin(curDates)].AGpos_PCRpos.values
# PCRposPct = dfPCR[dfPCR.Date.isin(curDates)].PosPct.values
PCRposPct = np.divide(PCRpos,numPCR)
AGposPct = np.divide(AGpos,numAG)

# Positive PCR-tests, excluding those that only got tested because of a positive AG-test
PCRpos_nonAG = PCRpos-AGposPCRpos

# Factor used for scaling with respect to number of tests
scaleFactor = 0.7

# Screening-ratio (Guess about how many tests would have been carried out in the absence of wide screening)
screenRatio = 1/6


scaleScreenRatio = screenRatio**scaleFactor

# 
multFactor = (1-screenRatio**scaleFactor)


fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

ax1.plot(curDates,AGposPCRpos,label='AG positive, PCR confirmed')
ax1.plot(curDates,PCRpos,label='PCR positive, total')
ax1.plot(curDates,PCRpos_nonAG,label='PCR positive, non-AG')
ax1.plot(curDates,np.multiply(PCRpos_nonAG,scaleScreenRatio),label='PCR positive, non-AG\nAssumed due to symptoms')
ax1.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),label='PCR positive, non-AG\nAssumed due to screening')
ax1.plot(curDates,AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),label='PCR positive, non-AG, screening,\nand AG confirmatory tests')

meanWidth = 7
ax1.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),meanWidth),'k:',label='PCR positive, non-AG, screening,\nand AG confirmatory tests, 7 day mean')
# ax1.plot(curDates,PCRpos - (AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)),label='PCR positive, as neighbours')

foundThroughScreening = AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)
foundThroughPCRScreening = np.multiply(PCRpos_nonAG,multFactor)


ax2.plot(curDates,100*PCRposPct,'b.-',linewidth=0.5,markersize=2,label='PCR, total')
ax2.plot(rnTime(curDates,meanWidth),100*rnMean(PCRposPct,meanWidth),'b',label='PCR, total, '+str(meanWidth)+' day mean')
ax2.plot(curDates,100*AGposPct,'.-',color='xkcd:dark red',label='AG',linewidth=0.5,markersize=2)
ax2.plot(rnTime(curDates,meanWidth),100*rnMean(AGposPct,meanWidth),color='xkcd:dark red',label='AG, '+str(meanWidth)+' day mean')
ax2.plot(curDates,2*100*AGposPct,'.-',color='xkcd:red',label='AG, multiplied by 2',linewidth=0.5,markersize=2)
ax2.plot(rnTime(curDates,meanWidth),2*100*rnMean(AGposPct,meanWidth),color='xkcd:red',label='AG, multiplied by 2, '+str(meanWidth)+' day mean')

ax2.plot(curDates,100*np.divide(foundThroughPCRScreening,numPCR*(1-screenRatio)),'.-',linewidth=0.5,markersize=2,color='xkcd:green',label='PCR, screening')
ax2.plot(rnTime(curDates,meanWidth),rnMean(100*np.divide(foundThroughPCRScreening,numPCR*(1-screenRatio)),meanWidth),color='xkcd:green',label='PCR, screening, '+str(meanWidth)+' day mean')
# ax2.plot(curDates,6*100*np.divide(PCRpos - (AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)),numPCR),label='PCR positive, as neighbours')

# ax2.plot(rnTime(curDates,meanWidth),100*rnMean(PCRposPct,meanWidth),'b')
# ax2.plot(rnTime(curDates,meanWidth),rnMean(6*100*np.divide(PCRpos - (AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)),numPCR),7),':')

ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax1.grid()
ax2.grid()
ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/EstimateOfEffectOfScreening')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)

meanWidth = 7

ax1.plot(curDates,AGposPCRpos,'.-',color='xkcd:hot pink',linewidth=0.5,markersize=2,label='AG positive, PCR confirmed')
ax1.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRpos,meanWidth),color='xkcd:hot pink',label=str(meanWidth)+' day running mean')
ax1.plot(curDates,PCRpos,'.-',color='xkcd:rust',linewidth=0.5,markersize=2,label='PCR positive, total')
ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos,meanWidth),color='xkcd:rust',label=str(meanWidth)+' day running mean')
ax1.plot(curDates,PCRpos_nonAG,'g.-',linewidth=0.5,markersize=2,label='PCR positive,\nexcluding tests to confirm AG-tests')
ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos_nonAG,meanWidth),'g',label=str(meanWidth)+' day running mean')


ax2.fill_between(curDates,np.multiply(PCRpos_nonAG,multFactor),PCRpos_nonAG,color='g',label='PCR positive,\nexcluding tests to confirm AG-tests')
ax2.fill_between(curDates,np.multiply(PCRpos_nonAG,multFactor),color='xkcd:royal blue',label=f'PCR positive, non-AG\nAssumed due to screening\n({100*multFactor:.2f} %)')

# ax1.plot(curDates,np.multiply(PCRpos_nonAG,scaleScreenRatio),label='PCR positive, non-AG\nAssumed due to symptoms')
# ax1.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),label='PCR positive, non-AG\nAssumed due to screening')
ax3.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),'.-',linewidth=0.5,markersize=2,color='xkcd:royal blue',label=f'PCR positive, non-AG\nAssumed due to screening\n({100*multFactor:.2f} %)')
ax3.plot(rnTime(curDates,meanWidth),rnMean(np.multiply(PCRpos_nonAG,multFactor),meanWidth),'xkcd:royal blue',label=str(meanWidth)+' day running mean')

ax3.plot(curDates,AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),'m.-',linewidth=0.5,markersize=2,label='PCR positive, non-AG, screening,\nand AG confirmatory tests')

ax3.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),meanWidth),'m',label=str(meanWidth)+' day running mean')

ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax3.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax1.grid()
ax3.grid()
ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)
ax3.set_ylim(bottom=0)



ax1.set_xlim([firstDate,lastDate-np.timedelta64(1,'D')])
plt.tight_layout()


if saveFigures:
    plt.savefig('figs/EstimateOfEffectOfScreeningNew')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
# fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)
fig,ax1 = plt.subplots(1,1,sharex=True)

meanWidth = 7

ax1.plot(curDates,PCRpos,'.-',color='xkcd:rust',linewidth=0.5,markersize=2,label='PCR positive, total')
ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos,meanWidth),color='xkcd:rust')
ax1.plot(curDates,PCRpos_nonAG,'g.-',linewidth=0.5,markersize=2,label='PCR positive, due to only PCR')
ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos_nonAG,meanWidth),'g')
ax1.plot(curDates,AGposPCRpos,'.-',color='xkcd:hot pink',linewidth=0.5,markersize=2,label='PCR positive, due to AG')
ax1.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRpos,meanWidth),color='xkcd:hot pink')


# ax2.fill_between(curDates,np.multiply(PCRpos_nonAG,multFactor),PCRpos_nonAG,color='g',label='PCR positive,\nexcluding tests to confirm AG-tests')
# ax2.fill_between(curDates,np.multiply(PCRpos_nonAG,multFactor),color='xkcd:royal blue',label=f'PCR positive, non-AG\nAssumed due to screening\n({100*multFactor:.2f} %)')

# # ax1.plot(curDates,np.multiply(PCRpos_nonAG,scaleScreenRatio),label='PCR positive, non-AG\nAssumed due to symptoms')
# # ax1.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),label='PCR positive, non-AG\nAssumed due to screening')
# ax3.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),'.-',linewidth=0.5,markersize=2,color='xkcd:royal blue',label=f'PCR positive, non-AG\nAssumed due to screening\n({100*multFactor:.2f} %)')
# ax3.plot(rnTime(curDates,meanWidth),rnMean(np.multiply(PCRpos_nonAG,multFactor),meanWidth),'xkcd:royal blue',label=str(meanWidth)+' day running mean')

# ax3.plot(curDates,AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),'m.-',linewidth=0.5,markersize=2,label='PCR positive, non-AG, screening,\nand AG confirmatory tests')

# ax3.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),meanWidth),'m',label=str(meanWidth)+' day running mean')

# ax1.legend(loc='center center',bbox_to_anchor = (0.5, 1.0))
ax1.legend(loc='upper left')
# ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
# ax3.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax1.grid()
# ax3.grid()
ax1.set_ylim(bottom=0,top=900)
# ax2.set_ylim(bottom=0)
# ax3.set_ylim(bottom=0)

ax1.set_ylabel('Count')


ax1.set_xlim([firstDate,lastDate-np.timedelta64(1,'D')])
plt.tight_layout()


if saveFigures:
    plt.savefig('Viggo_Screening')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
# fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)
fig,ax1 = plt.subplots(1,1,sharex=True)

meanWidth = 7

ax1.plot(curDates,PCRpos_nonAG,'g.-',linewidth=0.5,markersize=2,label='PCR positive, due to only PCR (all)')
ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos_nonAG,meanWidth),'g')


# ax2.fill_between(curDates,np.multiply(PCRpos_nonAG,multFactor),PCRpos_nonAG,color='g',label='PCR positive,\nexcluding tests to confirm AG-tests')
ax1.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),'.-',linewidth=0.5,markersize=2,color='xkcd:royal blue',label=f'PCR positive, due to only PCR (screening, {100*multFactor:.2f} %)')
ax1.plot(rnTime(curDates,meanWidth),rnMean(np.multiply(PCRpos_nonAG,multFactor),meanWidth),color='xkcd:royal blue')

# # ax1.plot(curDates,np.multiply(PCRpos_nonAG,scaleScreenRatio),label='PCR positive, non-AG\nAssumed due to symptoms')
# # ax1.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),label='PCR positive, non-AG\nAssumed due to screening')
# ax3.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),'.-',linewidth=0.5,markersize=2,color='xkcd:royal blue',label=f'PCR positive, non-AG\nAssumed due to screening\n({100*multFactor:.2f} %)')
# ax3.plot(rnTime(curDates,meanWidth),rnMean(np.multiply(PCRpos_nonAG,multFactor),meanWidth),'xkcd:royal blue',label=str(meanWidth)+' day running mean')

# ax3.plot(curDates,AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),'m.-',linewidth=0.5,markersize=2,label='PCR positive, non-AG, screening,\nand AG confirmatory tests')

# ax3.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),meanWidth),'m',label=str(meanWidth)+' day running mean')

# ax1.legend(loc='center center',bbox_to_anchor = (0.5, 1.0))
# ax1.legend()
ax1.legend(loc='upper left')
# ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
# ax3.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax1.grid()
# ax3.grid()
ax1.set_ylim(bottom=0,top=900)
# ax2.set_ylim(bottom=0)
# ax3.set_ylim(bottom=0)

ax1.set_ylabel('Count')


ax1.set_xlim([firstDate,lastDate-np.timedelta64(1,'D')])
plt.tight_layout()


if saveFigures:
    plt.savefig('Viggo_Screening2')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
# fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)
fig,ax1 = plt.subplots(1,1,sharex=True)

meanWidth = 7

# ax1.plot(curDates,PCRpos_nonAG,'g.-',linewidth=0.5,markersize=2,label='PCR positive, due to only PCR (all)')
# ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos_nonAG,meanWidth),'g')


# ax2.fill_between(curDates,np.multiply(PCRpos_nonAG,multFactor),PCRpos_nonAG,color='g',label='PCR positive,\nexcluding tests to confirm AG-tests')
ax1.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),'.-',linewidth=0.5,markersize=2,color='xkcd:royal blue',label=f'PCR positive, due to only PCR (screening, {100*multFactor:.2f} %)')
ax1.plot(rnTime(curDates,meanWidth),rnMean(np.multiply(PCRpos_nonAG,multFactor),meanWidth),color='xkcd:royal blue')

ax1.plot(curDates,AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),'m.-',linewidth=0.5,markersize=2,label=f'PCR positive, due to only PCR (screening, {100*multFactor:.2f} %) + AG confirmatory tests')

ax1.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),meanWidth),'m')


# # ax1.plot(curDates,np.multiply(PCRpos_nonAG,scaleScreenRatio),label='PCR positive, non-AG\nAssumed due to symptoms')
# # ax1.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),label='PCR positive, non-AG\nAssumed due to screening')
# ax3.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),'.-',linewidth=0.5,markersize=2,color='xkcd:royal blue',label=f'PCR positive, non-AG\nAssumed due to screening\n({100*multFactor:.2f} %)')
# ax3.plot(rnTime(curDates,meanWidth),rnMean(np.multiply(PCRpos_nonAG,multFactor),meanWidth),'xkcd:royal blue',label=str(meanWidth)+' day running mean')

# ax3.plot(curDates,AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),'m.-',linewidth=0.5,markersize=2,label='PCR positive, non-AG, screening,\nand AG confirmatory tests')

# ax3.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),meanWidth),'m',label=str(meanWidth)+' day running mean')

# ax1.legend(loc='center center',bbox_to_anchor = (0.5, 1.0))
# ax1.legend()
ax1.legend(loc='upper left')
# ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
# ax3.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax1.grid()
# ax3.grid()
ax1.set_ylim(bottom=0,top=900)
# ax2.set_ylim(bottom=0)
# ax3.set_ylim(bottom=0)

ax1.set_ylabel('Count')


ax1.set_xlim([firstDate,lastDate-np.timedelta64(1,'D')])
plt.tight_layout()


if saveFigures:
    plt.savefig('Viggo_Screening3')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
# firstDate = np.datetime64('2021-02-01')
# lastDate = np.datetime64('2021-04-24')

# curDates = np.arange(firstDate,lastDate)

# posAG  = df[df.Date.isin(curDates)].NewPositive 
PCRpos = dfPCR[dfPCR.Date.isin(curDates)].NewPositive.values
numPCR = dfPCR[dfPCR.Date.isin(curDates)].Tested.values
AGpos = df[df.Date.isin(curDates)].NewPositive.values
numAG = df[df.Date.isin(curDates)].Tested.values
AGposPCRpos = df2[df2.Dato.isin(curDates)].AGpos_PCRpos.values
# PCRposPct = dfPCR[dfPCR.Date.isin(curDates)].PosPct.values
PCRposPct = np.divide(PCRpos,numPCR)
AGposPct = np.divide(AGpos,numAG)

# Positive PCR-tests, excluding those that only got tested because of a positive AG-test
PCRpos_nonAG = PCRpos-AGposPCRpos

# Factor used for scaling with respect to number of tests
scaleFactor = 0.7

# Screening-ratio (Guess about how many tests would have been carried out in the absence of wide screening)
screenRatio = 1/6


scaleScreenRatio = screenRatio**scaleFactor

# 
multFactor = (1-screenRatio**scaleFactor)


fig,ax2 = plt.subplots(1,1,sharex=True)

# ax1.plot(curDates,AGposPCRpos,label='AG positive, PCR confirmed')
# ax1.plot(curDates,PCRpos,label='PCR positive, total')
# ax1.plot(curDates,PCRpos_nonAG,label='PCR positive, non-AG')
# ax1.plot(curDates,np.multiply(PCRpos_nonAG,scaleScreenRatio),label='PCR positive, non-AG\nAssumed due to symptoms')
# ax1.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),label='PCR positive, non-AG\nAssumed due to screening')
# ax1.plot(curDates,AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),label='PCR positive, non-AG, screening,\nand AG confirmatory tests')

meanWidth = 7
# ax1.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),meanWidth),'k:',label='PCR positive, non-AG, screening,\nand AG confirmatory tests, 7 day mean')
# ax1.plot(curDates,PCRpos - (AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)),label='PCR positive, as neighbours')

foundThroughScreening = AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)
foundThroughPCRScreening = np.multiply(PCRpos_nonAG,multFactor)


# ax2.plot(curDates,100*PCRposPct,'b.-',linewidth=0.5,markersize=2,label='PCR, total')
ax2.plot(rnTime(curDates,meanWidth),100*rnMean(PCRposPct,meanWidth),'b',label='PCR, total')
# ax2.plot(curDates,100*AGposPct,'.-',color='xkcd:dark red',label='AG',linewidth=0.5,markersize=2)
ax2.plot(rnTime(curDates,meanWidth),100*rnMean(AGposPct,meanWidth),color='xkcd:dark red',label='AG')
# ax2.plot(curDates,2*100*AGposPct,'.-',color='xkcd:red',label='AG, multiplied by 2',linewidth=0.5,markersize=2)
ax2.plot(rnTime(curDates,meanWidth),2*100*rnMean(AGposPct,meanWidth),'--',color='xkcd:red',label='AG, multiplied by 2')

# ax2.plot(curDates,100*np.divide(foundThroughPCRScreening,numPCR*(1-screenRatio)),'.-',linewidth=0.5,markersize=2,color='xkcd:green',label='PCR, screening')
ax2.plot(rnTime(curDates,meanWidth),rnMean(100*np.divide(foundThroughPCRScreening,numPCR*(1-screenRatio)),meanWidth),color='xkcd:green',label='PCR, screening')
# ax2.plot(curDates,6*100*np.divide(PCRpos - (AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)),numPCR),label='PCR positive, as neighbours')

# ax2.plot(rnTime(curDates,meanWidth),100*rnMean(PCRposPct,meanWidth),'b')
# ax2.plot(rnTime(curDates,meanWidth),rnMean(6*100*np.divide(PCRpos - (AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)),numPCR),7),':')

ax2.set_ylabel('Positive percentage [%]')
# ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='upper right')
# ax1.grid()
ax2.grid()
# ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=0.8)
ax2.set_xlim([firstDate,lastDate-np.timedelta64(1,'D')])
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/Viggo_pospct')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
# firstDate = np.datetime64('2021-02-01')
# lastDate = np.datetime64('2021-04-24')

# curDates = np.arange(firstDate,lastDate)

# posAG  = df[df.Date.isin(curDates)].NewPositive 
PCRpos = dfPCR[dfPCR.Date.isin(curDates)].NewPositive.values
numPCR = dfPCR[dfPCR.Date.isin(curDates)].Tested.values
AGpos = df[df.Date.isin(curDates)].NewPositive.values
numAG = df[df.Date.isin(curDates)].Tested.values
AGposPCRpos = df2[df2.Dato.isin(curDates)].AGpos_PCRpos.values
# PCRposPct = dfPCR[dfPCR.Date.isin(curDates)].PosPct.values
PCRposPct = np.divide(PCRpos,numPCR)
AGposPct = np.divide(AGpos,numAG)

# Positive PCR-tests, excluding those that only got tested because of a positive AG-test
PCRpos_nonAG = PCRpos-AGposPCRpos

# Factor used for scaling with respect to number of tests
scaleFactor = 0.7

# Screening-ratio (Guess about how many tests would have been carried out in the absence of wide screening)
screenRatio = 1/6


scaleScreenRatio = screenRatio**scaleFactor

# 
multFactor = (1-screenRatio**scaleFactor)


fig,ax2 = plt.subplots(1,1,sharex=True)


meanWidth = 7

foundThroughScreening = AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)
foundThroughPCRScreening = np.multiply(PCRpos_nonAG,multFactor)


# # ax2.plot(curDates,100*PCRposPct,'b.-',linewidth=0.5,markersize=2,label='PCR, total')
# ax2.plot(rnTime(curDates,meanWidth),100*rnMean(PCRposPct,meanWidth),'b',label='PCR, total')
# # ax2.plot(curDates,100*AGposPct,'.-',color='xkcd:dark red',label='AG',linewidth=0.5,markersize=2)
# ax2.plot(rnTime(curDates,meanWidth),100*rnMean(AGposPct,meanWidth),color='xkcd:dark red',label='AG')
# # ax2.plot(curDates,2*100*AGposPct,'.-',color='xkcd:red',label='AG, multiplied by 2',linewidth=0.5,markersize=2)
# ax2.plot(rnTime(curDates,meanWidth),2*100*rnMean(AGposPct,meanWidth),'--',color='xkcd:red',label='AG, multiplied by 2')

# ax2.plot(curDates,100*np.divide(foundThroughPCRScreening,numPCR*(1-screenRatio)),'.-',linewidth=0.5,markersize=2,color='xkcd:green',label='PCR, screening')
ax2.plot(rnTime(curDates,meanWidth),5800000*rnMean(np.divide(foundThroughPCRScreening,numPCR*(1-screenRatio)),meanWidth),color='xkcd:green',label='PCR, screening')
# ax2.plot(curDates,6*100*np.divide(PCRpos - (AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)),numPCR),label='PCR positive, as neighbours')

# ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='upper right')
# ax1.grid()
ax2.grid()
# ax2.set_ylim(bottom=0)
# ax2.set_ylim(bottom=0,top=0.8)
ax2.set_xlim([firstDate,lastDate-np.timedelta64(1,'D')])
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/Viggo_pospct2')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
# Positiv percentage of estimate
# First show false negative rate
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

# ax2.plot(curDates,100*PCRposPct,'b.-',linewidth=0.5,markersize=2,label='PCR, total')
# ax2.plot(rnTime(curDates,meanWidth),100*rnMean(PCRposPct,meanWidth),'b',label='PCR, total, '+str(meanWidth)+' day mean')
# ax2.plot(curDates,100*AGposPct,'.-',color='xkcd:dark red',label='AG',linewidth=0.5,markersize=2)
# ax2.plot(rnTime(curDates,meanWidth),100*rnMean(AGposPct,meanWidth),color='xkcd:dark red',label='AG, '+str(meanWidth)+' day mean')
# ax2.plot(curDates,2*100*AGposPct,'.-',color='xkcd:red',label='AG, multiplied by 2',linewidth=0.5,markersize=2)
# ax2.plot(rnTime(curDates,meanWidth),2*100*rnMean(AGposPct,meanWidth),color='xkcd:red',label='AG, multiplied by 2, '+str(meanWidth)+' day mean')

# ax2.plot(curDates,100*np.divide(foundThroughPCRScreening,numPCR*(1-screenRatio)),'.-',linewidth=0.5,markersize=2,color='xkcd:green',label='PCR, screening')
# ax2.plot(rnTime(curDates,meanWidth),rnMean(100*np.divide(foundThroughPCRScreening,numPCR*(1-screenRatio)),meanWidth),color='xkcd:green',label='PCR, screening, '+str(meanWidth)+' day mean')
# ax2.plot(curDates,6*100*np.divide(PCRpos - (AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)),numPCR),label='PCR positive, as neighbours')

# ax2.plot(rnTime(curDates,meanWidth),100*rnMean(PCRposPct,meanWidth),'b')
# ax2.plot(rnTime(curDates,meanWidth),rnMean(6*100*np.divide(PCRpos - (AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)),numPCR),7),':')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Age stuff

In [25]:
dfAge = pd.read_csv(latestdir+'/Cases_by_age.csv',delimiter = ';',dtype=str)

In [26]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath


conf = []
test = []
posp = []

for subdir, dirs, files in os.walk(rootdir):
    if not len(files) == 0:
        latestdir = subdir
        latestDate = pd.to_datetime(subdir[-10:])
        curdf = pd.read_csv(latestdir+'/Cases_by_age.csv',delimiter = ';',dtype=str)
        conf.append(curdf.iloc[9,1])
        test.append(curdf.iloc[9,2])
        posp.append(curdf.iloc[9,3])
        # print(curdf)

In [27]:
# dfAge = pd.read_csv''/Cases_by_age.csv',delimiter = ';',dtype=str)
latestdir

dfAge = pd.read_csv('ssi_data\\SSI_data_2021-04-06\\Cases_by_age.csv',delimiter = ';',dtype=str)

In [28]:
dfAge

,Aldersgruppe,Antal_bekræftede_COVID-19,Antal_testede,Procent_positive
0,0-9,17.004,411.910,"4,1"
1,10-19,36.065,608.757,"5,9"
2,20-29,43.793,679.566,"6,4"
3,30-39,32.409,596.068,"5,4"
4,40-49,34.396,637.766,"5,4"
5,50-59,33.504,651.292,"5,1"
6,60-69,18.247,487.720,"3,7"
7,70-79,11.219,369.509,"3,0"
8,80-89,5.850,140.967,"4,1"
9,90+,1.830,29.111,"6,3"


In [29]:
# dfDeaths = pd.read_excel("C://Users//rakrpe//Documents//PandemiX//GithubRepos//PandemiX//DanskeData//DeathsAgeDK//Deaths.xlsx")

In [30]:
dfDeaths = pd.read_excel('DeathsAgeDK/Deaths.xlsx')

In [31]:
dfDeaths = dfDeaths.transpose()

In [32]:
dfDeaths.columns = dfDeaths.iloc[0]

In [33]:
dfDeaths = dfDeaths.drop(['Aldersgruppe']) 

In [34]:
# plt.figure()
# plt.plot(dfDeaths)


curDates =  pd.to_datetime(dfDeaths.index,format='%d_%m_%Y')

In [35]:
# plt.figure()
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)

# for colNames in dfDeaths.columns[:-1]:
for colNames in dfDeaths.columns:
    # plt.plot(curDates,dfDeaths['0-9'])
    ax1.plot(curDates,dfDeaths[colNames],label=colNames)
    ax2.plot(curDates[0:-1],np.diff(dfDeaths[colNames]),label=colNames)
    ax3.plot(curDates[0:-1],np.diff(dfDeaths[colNames]),label=colNames)

ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax1.grid()
ax2.grid()
ax3.grid()

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)
ax3.set_ylim(bottom=0,top=10)

# ax1.set_xlim(left=curDates[0],right=date.today())
ax1.set_xlim(right=date.today())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d %b\n%Y'))

ax1.set_ylabel('Kumulative dødsfald')
ax2.set_ylabel('Dødsfald')
ax3.set_ylabel('Dødsfald')

plt.tight_layout()

if saveFigures:
    fig.savefig('figs/DeathsByAge')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
# plt.figure()
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
fig,ax1 = plt.subplots(1,1,sharex=True)

curVals = np.zeros(curDates[0:-1].shape)
for colNames in dfDeaths.columns:

    curVals += np.diff(dfDeaths[colNames])
    ax1.plot(curDates[0:-1],curVals)

    # plt.plot(curDates,dfDeaths['0-9'])
    # ax1.plot(curDates,dfDeaths[colNames],label=colNames)

    # ax2.plot(curDates[0:-1],np.diff(dfDeaths[colNames]),label=colNames)
    # ax3.plot(curDates[0:-1],np.diff(dfDeaths[colNames]),label=colNames)

ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax1.grid()

ax1.set_ylim(bottom=0)

# ax1.set_xlim(left=curDates[0],right=date.today())
ax1.set_xlim(right=date.today())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d %b\n%Y'))

ax1.set_ylabel('Kumulative dødsfald')
# ax2.set_ylabel('Dødsfald')
# ax3.set_ylabel('Dødsfald')

plt.tight_layout()

# if saveFigures:
#     fig.savefig('figs/DeathsByAge')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

TypeError: ufunc 'add' output (typecode 'O') could not be coerced to provided output parameter (typecode 'd') according to the casting rule ''same_kind''

In [ ]:
# np.diff(dfDeaths[colNames])
# curVals
dfDeaths

In [ ]:
dfAge

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)


ax1.plot(curDates,dfDeaths['60-69'])

plt.tight_layout()

In [ ]:
curDates